# FarmerFriendly ML Model Conversion

This notebook converts your Keras model and pickle scalers to formats that work with Supabase Edge Functions.

## Steps:
1. Run the first cell to install dependencies
2. Upload your 3 files when prompted
3. Run the conversion cells
4. Download the converted files
5. Upload them to Supabase Storage

In [ ]:
# Install required packages
!pip install tensorflowjs tensorflow scikit-learn -q
print("✅ Dependencies installed!")

In [ ]:
# Upload your files
from google.colab import files
import os

print("📤 Please upload your 3 files:")
print("   - purair_model.keras")
print("   - scaler_X.pkl")
print("   - scaler_y.pkl")
print()

uploaded = files.upload()

# Verify files
expected_files = ['purair_model.keras', 'scaler_X.pkl', 'scaler_y.pkl']
uploaded_names = list(uploaded.keys())

print(f"\n✅ Uploaded {len(uploaded_names)} file(s):")
for name in uploaded_names:
    print(f"   - {name}")

In [ ]:
# Convert Keras model to TensorFlow.js format
import tensorflowjs as tfjs
from tensorflow import keras

print("🔄 Converting Keras model to TensorFlow.js...")

# Find the keras model file
keras_file = [f for f in uploaded_names if f.endswith('.keras')][0]
print(f"   Loading {keras_file}...")

model = keras.models.load_model(keras_file)
print(f"   Model loaded successfully!")
print(f"   Model input shape: {model.input_shape}")
print(f"   Model output shape: {model.output_shape}")

# Convert to TensorFlow.js
os.makedirs('tfjs_model', exist_ok=True)
tfjs.converters.save_keras_model(model, 'tfjs_model')

print("\n✅ Model converted successfully!")
print("   Files created in tfjs_model/:")
for f in os.listdir('tfjs_model'):
    print(f"   - {f}")

In [ ]:
# Convert scalers to JSON format
import pickle
import json
import numpy as np

print("🔄 Converting scalers to JSON format...\n")

# Load scaler_X
with open('scaler_X.pkl', 'rb') as f:
    scaler_X = pickle.load(f)

scaler_X_params = {
    'mean': scaler_X.mean_.tolist(),
    'scale': scaler_X.scale_.tolist()
}

with open('scaler_X.json', 'w') as f:
    json.dump(scaler_X_params, f, indent=2)

print("✅ scaler_X.json created")
print(f"   Mean: {scaler_X_params['mean']}")
print(f"   Scale: {scaler_X_params['scale']}")

# Load scaler_y
with open('scaler_y.pkl', 'rb') as f:
    scaler_y = pickle.load(f)

scaler_y_params = {
    'mean': scaler_y.mean_.tolist(),
    'scale': scaler_y.scale_.tolist()
}

with open('scaler_y.json', 'w') as f:
    json.dump(scaler_y_params, f, indent=2)

print("\n✅ scaler_y.json created")
print(f"   Mean: {scaler_y_params['mean']}")
print(f"   Scale: {scaler_y_params['scale']}")

print("\n✅ All conversions complete!")

In [ ]:
# Download all converted files
import shutil

print("📦 Preparing files for download...\n")

# Create a zip file with all converted files
shutil.make_archive('converted_model', 'zip', 'tfjs_model')

print("📥 Download these files:\n")

# Download the model zip
files.download('converted_model.zip')
print("   ✅ converted_model.zip (contains model.json and weight files)")

# Download scalers
files.download('scaler_X.json')
print("   ✅ scaler_X.json")

files.download('scaler_y.json')
print("   ✅ scaler_y.json")

print("\n" + "="*60)
print("🎉 CONVERSION COMPLETE!")
print("="*60)
print("\nNext steps:")
print("1. Extract converted_model.zip")
print("2. Go to: https://supabase.com/dashboard/project/ipfqxgzylmfyffeqpdrb/storage/buckets/models")
print("3. Upload ALL files from the zip (model.json and all .bin files)")
print("4. Upload scaler_X.json")
print("5. Upload scaler_y.json")
print("\n✅ Then your ML predictions will work!")

## Troubleshooting

If you get errors:

1. **"No module named 'tensorflowjs'"** - Re-run the first cell
2. **"File not found"** - Make sure you uploaded all 3 files in step 2
3. **Model loading errors** - Check that purair_model.keras is a valid Keras model
4. **Scaler errors** - Verify scaler files are StandardScaler pickle files

After uploading to Supabase, test by going to your Analytics page and clicking "Generate New Prediction"!